# Getting data into a graph, and results out

Now that we know PyRaphtory is installed and running, let's look at the different ways to get some real data into a graph. 

For this first set of tutorials we are going to be building graphs from a Lord of the Rings 🧝🏻‍♀️🧙🏻‍♂️💍 dataset, looking at when characters interact throughout the trilogy.
 
As with the quick start install guide, this and all following python pages are built as iPython notebooks. If you want to follow along on your own machine, click the `open on github` link in the top right of this page.


<p>
 <img src="../_static/lotr-graphic.png" width="700px" style="padding: 15px" alt="Intro Graphic of LOTR slices"/>
</p>

## Let's have a look at the example data

The data is a `csv` file (comma-separated values) and is pulled from our <a href="https://github.com/Raphtory/Data/blob/main/lotr.csv" target="_blank">Github data repository</a>. Each line contains two characters that appeared in the same sentence, along with the sentence number, which we will use as a `timestamp`. The first line of the file is `Gandalf,Elrond,33` which tells
us that Gandalf and Elrond appears together in sentence 33.

### Downloading the csv from Github 💾

The following `curl` command will download the csv file and save it in the `tmp` directory on your computer. This will be deleted when you restart your computer, but it's only a couple of KB in any case.





In [1]:
!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv
!head /tmp/lotr.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   201k      0 --:--:-- --:--:-- --:--:--  201k
Gandalf,Elrond,33
Frodo,Bilbo,114
Blanco,Marcho,146
Frodo,Bilbo,205
Thorin,Gandalf,270
Thorin,Bilbo,270
Gandalf,Bilbo,270
Gollum,Bilbo,286
Gollum,Bilbo,306
Gollum,Bilbo,308


## Ingesting data with in-built Sources

As this data is in an `edge list` format, the easiest way to get your data into a graph is to use our automatic 

In [ ]:
with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        source_node = row[0]
        src_id = graph.assign_id(source_node)
        target_node = row[1]
        tar_id = graph.assign_id(target_node)
        time_stamp = int(row[2])
        graph.add_vertex(time_stamp, src_id, Properties(ImmutableProperty("name", source_node)), Type("Character"))
        graph.add_vertex(time_stamp, tar_id, Properties(ImmutableProperty("name", target_node)), Type("Character"))
        graph.add_edge(time_stamp, src_id, tar_id, Type("Character_Co-occurence"))

|